In [1]:
from pyrosetta import *
from rosetta import *
import pandas as pd


/var/folders/td/b3wzd0yj2rdb8m8jzhlf7ykw0000gn/T/ipykernel_58683/3550713.py:2: UserWarning: Import of 'rosetta' as a top-level module is deprecated and may be removed in 2018, import via 'pyrosetta.rosetta'.
  from rosetta import *


In [2]:
from mutationscoring import g_chillaxer
from scanning import get_scanner


┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.Release.python313.m1 2025.06+release.029c6a159b896477003a14f78f472d4cd2cead46 2025-02-04T15:14:13] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python313.m1 r394 2025.06+release.029c6a159b 029c6a159b89647700

In [3]:
complex_pose = Pose()
complex_p = pose_from_file("../peptidetrial.pdb")
chain = 'B'
radius_relax = 10
positions = [1]


core.chemical: [ WARNING ] protein residue ETH is not explicitly listed as either L or D in its params file.
core.chemical: [ WARNING ] To avoid seeing this warning in the future, add "L_AA", "D_AA", or "ACHIRAL_BACKBONE" to the "PROPERTIES" line of the params file.
core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 1020 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.757773 seconds.
core.import_pose.import_pose: File '../peptidetrial.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] skipping pdb residue b/c it's missing too many mainchain atoms:    0 B ALA ALA:NtermProteinFull
core.io.pose_from_sfr.PoseFromSFRBuilder: missing:  N
core.io.pose_from_sfr.PoseFromSFRBuilder: Adding undetected lower terminus type to residue 119,    1 B
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE  on residue LYS 8
core.conformation.Conformation: [ WARNING ] mi

In [4]:
# ncaa_array = pd.read_csv("../ncaa.csv", header=None)
# ncaas = ncaa_array[1].tolist()[1:]
# ncaa_names = ncaa_array[0].tolist()[1:]

ncaas = ['00R', 'A06']
ncaa_names = ['insetSmiles', 'insertSmiles']


In [5]:

trials = range(5)
mutation_relax = False
mutation_dump = False

In [6]:
relaxed_wildtype = g_chillaxer(complex_p, chain)

protocols.relax.RelaxScriptManager: Reading relax scripts list from database.
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
protocols.relax.RelaxScriptManager: Looking for MonomerRelax2019.txt
protocols.relax.RelaxScriptManager: ================== Reading script file: /Users/nathanphan/anaconda3/envs/ncaa/lib/python3.13/site-packages/pyrosetta/database/sampling/relax_scripts/MonomerRelax2019.txt ==================
protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
protocols.relax.RelaxScriptManager: coord_cst_weight 1.0
protocols.relax.RelaxScriptManager: scale:fa_rep 0.040
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.051
protocols.relax.RelaxScriptManager: min 0.01
protocols.relax.RelaxScriptManager: coord_cst_weight 0.5
protocols.relax.RelaxScriptManager: scale:fa_rep 0.265
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.280
protocols.relax.RelaxScriptManager: min 0.01
prot

In [7]:
large_op = []
for i in range(len(ncaas)):
    ncaa = ncaas[i]
    smiles = ncaa_names[i]
    print(f"Processing {ncaa} ({ncaa})")
    try:
        scanner = get_scanner(complex_p, chain, positions, ncaa, smiles, radius_relax, trials, mutationrelx = False, mutation_dump = False)

        df = scanner.score(complex_p, chain, positions, ncaa, radius_relax)
        print(df)
        large_op.append(df)
        print(large_op)
        fdf = pd.concat(large_op, ignore_index=True)
        print('==' * 20)
        print(fdf)
        if mutation_dump:
            scanner.dump_mutations()
        
        if mutation_relax:
            relaxed_pose = g_chillaxer(scanner.pose, chain)
            relaxed_pose.dump_pdb(f"{ncaa}_relaxed.pdb")
    except Exception as e:
        print(f"Error processing {ncaa}: {e}")
        continue
    print(fdf.head())

    # df.to_csv('../score.csv')
    # print(df.head())

Processing 00R (00R)
core.chemical.AtomICoor: [ WARNING ] IcoorAtomID::atom_id(): Cannot get atom_id for POLYMER_LOWER of residue 00R 118.  Returning BOGUS ID instead.
core.conformation.Residue: [ WARNING ] missing an atom: 118  H   that depends on a nonexistent polymer connection!
core.conformation.Residue: [ WARNING ]  --> generating it using idealized coordinates.
core.pack.rotamers.SingleNCAARotamerLibraryCreator: Reading in rot lib /Users/nathanphan/anaconda3/envs/ncaa/lib/python3.13/site-packages/pyrosetta/database//rotamer/ncaa_rotlibs/ncaa_rotamer_libraries/alpha_amino_acid/00R_rotamer.pdb.rotlib...
core.chemical.AtomICoor: [ WARNING ] IcoorAtomID::atom_id(): Cannot get atom_id for POLYMER_LOWER of residue 00R 118.  Returning BOGUS ID instead.
core.conformation.Residue: [ WARNING ] missing an atom: 118  H   that depends on a nonexistent polymer connection!
core.conformation.Residue: [ WARNING ]  --> generating it using idealized coordinates.
core.chemical.AtomICoor: [ WARNING ]

In [ ]:
print(fdf)

# df.columns = ["Position", "Trial", "WT Sequence", "Mutant Sequence", "Target Sequence","NCAA", "ddG", "Backbone Delta", "Score Difference", "NCAA String"]

  Position Trial    WT Sequence Mutant Sequence  \
0        1     0  FLFVIRDRVFRCG   XLFVIRDRVFRCG   
1        1     1  FLFVIRDRVFRCG   XLFVIRDRVFRCG   
2        1     2  FLFVIRDRVFRCG   XLFVIRDRVFRCG   
3        1     3  FLFVIRDRVFRCG   XLFVIRDRVFRCG   
4        1     4  FLFVIRDRVFRCG   XLFVIRDRVFRCG   
5        1     0  FLFVIRDRVFRCG   XLFVIRDRVFRCG   
6        1     1  FLFVIRDRVFRCG   XLFVIRDRVFRCG   
7        1     2  FLFVIRDRVFRCG   XLFVIRDRVFRCG   
8        1     3  FLFVIRDRVFRCG   XLFVIRDRVFRCG   
9        1     4  FLFVIRDRVFRCG   XLFVIRDRVFRCG   

                                     Target Sequence NCAA  \
0  AFTVTVPKDLYVVEYGSNMTIECKFPVEKQLDLAALIVYWEMEDKN...  00R   
1  AFTVTVPKDLYVVEYGSNMTIECKFPVEKQLDLAALIVYWEMEDKN...  00R   
2  AFTVTVPKDLYVVEYGSNMTIECKFPVEKQLDLAALIVYWEMEDKN...  00R   
3  AFTVTVPKDLYVVEYGSNMTIECKFPVEKQLDLAALIVYWEMEDKN...  00R   
4  AFTVTVPKDLYVVEYGSNMTIECKFPVEKQLDLAALIVYWEMEDKN...  00R   
5  AFTVTVPKDLYVVEYGSNMTIECKFPVEKQLDLAALIVYWEMEDKN...  A06   
6  AFTVTVPK